### 使用mongoexport 拉出mongodb 資料

In [6]:
# cd desktop
# mongoexport --host 10.2.18.6 --db userstar --collection userstar --out readmoo_all.json

### 將readmoo的書評 做資料清洗

In [5]:
import pandas as pd
def etl(filename):
    df=pd.read_json(filename,lines=True)
    df=df.drop(df[df["ISBN"]=='0'].index,axis=0)
    df=df.drop(df[df['ISBN']==''].index,axis=0)
    df=df.drop(df[df['USER'].isnull()].index,axis=0)
    df=df.drop(df[df['ISBN']==''].index,axis=0)
    df=df.dropna()    
    df['ISBN'] = df['ISBN'].astype('str')  # 確認 ISBN 為 STR
    df['USERSTAR'] = df['USERSTAR'].astype('float64')
    del df['_id']

    pattern = '|'.join(['\n', '\r', '\t','\xa0','\u3000'])
    df['CONTENT'] = df['CONTENT'].str.replace(pattern, '') 
    df=df.reset_index(drop=True)
    print(df.count())
    return df
#     df.to_json('cleaned_comment.json',orient='records',lines=True,force_ascii=False)
filename = r'C:\Users\Tibame_T14\Desktop\readmoo_all.json'
readmoo_df=etl(filename)

<ipython-input-5-30ece8e93003>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['CONTENT'] = df['CONTENT'].str.replace(pattern, '')


ISBN        100176
BOOKNAME    100176
USER        100176
CONTENT     100176
USERSTAR    100176
dtype: int64


### 將kingstone書評做清洗

In [6]:
# combine dataframe from folder
from functools import reduce
import pandas as pd
import os
def kingstone(filename):

    df_kingstone=pd.read_json(filename,lines=True)
    df_kingstone=df_kingstone.drop(df_kingstone[df_kingstone['ISBN']==''].index,axis=0)
    df_kingstone=df_kingstone.drop(df_kingstone[df_kingstone["ISBN"]=='0'].index,axis=0)
    df_kingstone=df_kingstone.drop(df_kingstone[df_kingstone['USER'].isnull()].index,axis=0)

    df_kingstone=df_kingstone.dropna()
    df_kingstone['ISBN'] = df_kingstone['ISBN'].astype('str')  # 確認 ISBN 為 STR

#     del readmoo_df['_id']

    pattern = '|'.join(['\n', '\r', '\t','\xa0','\u3000'])
    df_kingstone['CONTENT'] = df_kingstone['CONTENT'].str.replace(pattern, '') 
    df_kingstone=df_kingstone.reset_index(drop=True)
    df_kingstone['USERSTAR']=float(0)
    print(df_kingstone.count())
    return df_kingstone
filename = r'king_comment.json'
kingstone_df=kingstone(filename)

BOOKNAME    2515
ISBN        2515
USER        2515
CONTENT     2515
USERSTAR    2515
dtype: int64


<ipython-input-6-16e8fe3faec1>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df_kingstone['CONTENT'] = df_kingstone['CONTENT'].str.replace(pattern, '')


### 將金石堂書籍做清洗

In [2]:
import pandas as pd

In [27]:
df=pd.read_json('./test_not_send/test.json',lines=True)

In [28]:
del df['_id']

In [29]:
df=df.drop_duplicates(subset=['ISBN'])
df=df.drop(df[df["ISBN"]=='0'].index,axis=0)
df=df.drop(df[df['書籍簡介']=='0'].index,axis=0)
df=df.drop(df[df['書籍簡介']==''].index,axis=0)
df=df.drop(df[df['ISBN']==''].index,axis=0)

In [30]:
pattern = '|'.join(['\n', '\r', '\t','\xa0','\u3000'])
df['書籍簡介'] = df['書籍簡介'].str.replace(pattern, '')

/var/folders/9x/8jkp3nzj0sj4g7w0s98xq_2m0000gn/T/ipykernel_1064/1920646092.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['書籍簡介'] = df['書籍簡介'].str.replace(pattern, '')


In [32]:
df=df.dropna(axis=0)
df=df.reset_index(drop=True)

In [33]:
df.count()

書名      472834
書籍網站    472834
作者      472834
出版社     472834
ISBN    472834
圖片網址    472834
書籍簡介    472834
dtype: int64

In [ ]:
df.to_json('all_books.json',orient='records',lines=True,force_ascii=False)

### 清洗書籍簡介 column
columns='書名','ISBN','words'

In [34]:
df['words'] = df['書名']+'，' + df['書籍簡介']

In [35]:
del df['書籍網站'],df['作者'],df['出版社'],df['圖片網址'],df['書籍簡介']

In [36]:
stop_words_list=[]
with open(file='./test_not_send/stop_words.txt',mode='r', encoding="UTF-8") as file:
    for line in file:
        line = line.strip()
        stop_words_list.append(line)

In [37]:
stop_words='|'.join(stop_words_list)

In [38]:
df['words']=df.words.str.replace(stop_words,'')

/var/folders/9x/8jkp3nzj0sj4g7w0s98xq_2m0000gn/T/ipykernel_1064/92872001.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['words']=df.words.str.replace(stop_words,'')


In [ ]:
df.to_json('./test_not_send/intro_for_jieba.json',orient='records',lines=True,force_ascii=False)

### 對簡介斷詞( py上運行 )

In [ ]:
from multiprocessing import Pool
import pandas as pd
import jieba
import jieba.analyse
import time
df=pd.read_json('./test_not_send/intro_for_jieba.json',lines=True)
jieba.set_dictionary('test_not_send/dict.txt.big')

df_store=df.to_dict('record')
need=df_store[:4000]

def cut_word(dict1):
    cw=jieba.cut(dict1['words'])
    result=' '.join(cw)
    tags=jieba.analyse.extract_tags(result,topK=40,allowPOS=('n','ns','nz','v'))
    dict1['jieba_cut']=','.join(tags)
if __name__ == '__main__':
    start_time=time.time()
    with Pool(4) as p:
        p.map(cut_word,need)
    end_time=time.time()
    print(end_time-start_time)
    
    df_need=pd.Dataframe(need)
    df_need.to_json('./test_not_send/jieba_top40.json',orient='records',lines=True,force_ascii=False)

### 回頭清洗ISBN
(演算法結果發現ISBN不為數值)

In [39]:
df=pd.read_json('./test_not_send/jieba_top40.json',lines=True)

In [42]:
df_origin= pd.read_json('./test_not_send/categ.json',lines=True)

In [41]:
del df['_id']

In [44]:
df.loc[df['ISBN'].str.contains('http')][:].head()

,書名,ISBN,jieba_cut
15334,Compiler The Greed Whisperer,https://cdn.kingstone.com.tw/english/images/pr...,
15335,Compiler The Greed Whisperer,https://cdn.kingstone.com.tw/english/images/pr...,
15336,Introduction to Compiler Design,https://cdn.kingstone.com.tw/english/images/pr...,
15337,Introduction to Compiler Design,https://cdn.kingstone.com.tw/english/images/pr...,
15338,A PHP Compiler for the .NET Platform,https://cdn.kingstone.com.tw/english/images/pr...,


In [43]:
df_origin.count()

書名      472834
書籍網站    472834
作者      472834
出版社     472834
ISBN    472834
圖片網址    472834
書籍簡介    472834
類別      235057
dtype: int64